<h1>Imports</h1>

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn as slearn

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import svm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from imblearn.combine import SMOTETomek

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,ExtraTreesClassifier,GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn import tree
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from  sklearn.ensemble import RandomForestRegressor

Using TensorFlow backend.


In [2]:
pd.set_option("display.max_columns", 100)
%matplotlib inline

In [3]:
RANDOM_SEED = 6

<h1>Dataset Importing</h1>

In [4]:
DATA_PATH = Path.cwd() / "../../datasets"

In [5]:
training_df = pd.read_csv(DATA_PATH / "train.csv", index_col="tripid")
training_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [6]:
training_df = training_df.dropna(subset=['fare'])

<h1>Data Preprocessing</h1>

In [7]:
training_df['pickup_time'] = pd.to_datetime(training_df['pickup_time'], format="%m/%d/%Y %H:%M")
training_df['drop_time'] = pd.to_datetime(training_df['drop_time'], format="%m/%d/%Y %H:%M")

In [8]:
training_df = training_df.assign(timeOfDay=pd.cut(training_df.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [9]:
def dist_from_coordinates(lat1, lon1, lat2, lon2):
  R = 6371  # Earth radius in km

  #conversion to radians
  d_lat = np.radians(lat2-lat1)
  d_lon = np.radians(lon2-lon1)

  r_lat1 = np.radians(lat1)
  r_lat2 = np.radians(lat2)

  #haversine formula
  a = np.sin(d_lat/2.) **2 + np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2

  haversine = 2 * R * np.arcsin(np.sqrt(a))

  return haversine

In [10]:
new_column = []                    #empty column for distance
for index,row in training_df.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

training_df.insert(4,"distance",new_column)

In [11]:
training_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,distance,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,timeOfDay
tripid,,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.000000,5.092770,64.0,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,6.90330,79.8783,270.32,correct,dawn
189125358,10.5,791.0,47.0,0.000000,3.168058,134.0,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,6.91373,79.8923,197.85,correct,dawn
189125719,10.5,1087.0,80.0,0.000000,6.305395,61.0,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,6.93669,79.9146,301.64,correct,dawn
189127273,10.5,598.0,271.0,15.663800,0.861946,68.0,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,6.92748,79.8971,82.30,correct,dawn
189128020,NaN,NaN,NaN,NaN,8.147782,NaN,2019-11-01 03:34:00,2019-11-01 03:51:00,6.87441,79.8615,6.84478,79.9290,358.39,correct,dawn
189129552,10.5,3407.0,182.0,0.000000,24.207039,112.0,2019-11-01 05:38:00,2019-11-01 06:35:00,7.13402,79.8969,6.91865,79.8649,1065.02,correct,dawn
189132829,10.5,1246.0,487.0,0.000000,4.777624,133.0,2019-11-01 06:29:00,2019-11-01 06:49:00,6.84371,79.9051,6.85069,79.8624,266.62,correct,dawn
189135103,10.5,1333.0,295.0,17.198500,5.322544,212.0,2019-11-01 06:50:00,2019-11-01 07:12:00,6.90760,79.9524,6.90634,79.9042,318.05,correct,dawn
189139296,10.5,360.0,80.0,4.664000,1.035302,3.0,2019-11-01 07:00:00,2019-11-01 07:06:00,7.26706,80.6064,7.27422,80.6124,100.32,correct,dawn


In [12]:
correct_training_df = training_df[training_df['label'] == 'correct']

In [13]:
features_predictor = correct_training_df[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','distance','timeOfDay']]
fare_prediction = correct_training_df[['fare']]

In [14]:
features_predictor.dtypes

additional_fare               float64
duration                      float64
meter_waiting                 float64
meter_waiting_fare            float64
meter_waiting_till_pickup     float64
distance                      float64
timeOfDay                    category
dtype: object

In [15]:
numerical_features = features_predictor.columns[features_predictor.dtypes == "float64"].values
categorical_features = features_predictor.columns[features_predictor.dtypes == "category" ].values

In [16]:
X_train, X_eval, y_train, y_eval = train_test_split(features_predictor, fare_prediction, test_size=0.3, shuffle=True)

<h1>Data Cleaning</h1>

<h1>Feature Engineering</h1>

<h1>Model Training</h1>

In [17]:

## chain numerical preprocessing into a pipeline object
numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='median'))
])

non_numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore')),
#     ('label_encoder', LabelEncoder())
])

## create preprocessor stage of the final pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('non_numeric', non_numeric_preprocessing_steps, categorical_features),
        ('numeric', numeric_preprocessing_steps, numerical_features)
    ],
    remainder = 'drop'
)

In [18]:
new_columns = ['isDawn','isDay','isNight','additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup', 'distance']

In [19]:
preprocessed_train_features = preprocessor.fit_transform(X_train)
preprocessed_train_features_data_frame = pd.DataFrame(data=preprocessed_train_features, columns=new_columns)

In [20]:
preprocessed_test_features = preprocessor.fit_transform(X_eval)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_test_features, columns=new_columns)

In [ ]:
estimator = RandomForestRegressor()
## Train the model
estimator.fit(preprocessed_train_features_data_frame, y_train)

# Predict for the evaluation set

y_pred = estimator.predict(preprocessed_test_features_data_frame)
slearn.metrics.mean_squared_error(y_pred, y_eval)

In [ ]:
importances = estimator.feature_importances_
std = np.std([tree.feature_importances_ for tree in estimator.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(preprocessed_train_features.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(preprocessed_train_features.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(preprocessed_train_features.shape[1]), indices)
plt.xlim([-1, preprocessed_train_features.shape[1]])
plt.show()

In [ ]:
estimator2 = LinearRegression()
## Train the model
estimator2.fit(preprocessed_train_features, y_train)

# Predict for the evaluation set

y_pred = estimator2.predict(preprocessed_test_features_data_frame)
slearn.metrics.mean_squared_error(y_pred, y_eval)

In [ ]:
required_features = training_df[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','distance','timeOfDay']]
preprocessed_full_features = preprocessor.fit_transform(required_features)
preprocessed_full_features_data_frame = pd.DataFrame(data=preprocessed_full_features, columns=new_columns)

In [ ]:
predicted_prices = estimator.predict(preprocessed_full_features_data_frame)

In [ ]:
preprocessed_full_features_data_frame

In [ ]:
predicted_prices

In [ ]:
training_df.insert(10,'predicted_price',predicted_prices)

In [ ]:
classifier_features = training_df[['predicted_price','fare']]
classifier_label = training_df[['label']]
mapping = {'correct': 1, 'incorrect': 0}
classifier_label = classifier_label.replace({'label':mapping})

In [ ]:
classifier_features.isna().sum()

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(classifier_features, classifier_label, test_size=0.3, shuffle=True)

In [ ]:
smk = SMOTETomek(random_state=42)
os_x_train, os_y_train = smk.fit_sample(X_train,y_train)

In [ ]:

classifier = MLPClassifier()
## Train the model
classifier.fit(os_x_train, os_y_train)

# Predict for the evaluation set
print("Training Accuracy: %.2f" % (classifier.score(X_eval, y_eval)*100), "%")
y_pred = classifier.predict(X_eval)

In [ ]:
confusion_matrix = confusion_matrix(y_eval, y_pred)
confusion_matrix

In [ ]:
print(classification_report(y_eval, y_pred))

<h1>Model Validation</h1>

In [ ]:
test_set = pd.read_csv(DATA_PATH / "test.csv", index_col="tripid")
test_set.head()

In [ ]:
test_set.isna().sum()

In [ ]:
test_set['pickup_time'] = pd.to_datetime(test_set['pickup_time'], format="%m/%d/%Y %H:%M")
test_set['drop_time'] = pd.to_datetime(test_set['drop_time'], format="%m/%d/%Y %H:%M")

In [ ]:
test_set = test_set.assign(timeOfDay=pd.cut(test_set.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [ ]:
new_column = []                    #empty column for distance
for index,row in test_set.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

test_set .insert(4,"distance",new_column)

In [ ]:
test_set

In [ ]:
test_features = test_set[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','distance','timeOfDay']]
preprocessed_test_features = preprocessor.fit_transform(test_features)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_test_features, columns=new_columns)

In [ ]:
test_probs = estimator.predict(preprocessed_test_features_data_frame)

In [ ]:
test_set.insert(10,'predicted_price',test_probs)

In [ ]:
classifier_test_features = test_set[['predicted_price','fare']]

In [ ]:
predicted_labels = classifier.predict(classifier_test_features)

In [ ]:
submission_set = pd.read_csv(DATA_PATH / "sample_submission.csv", index_col="tripid")
submission_set.head()

submission_set['prediction']= predicted_labels

In [ ]:
submission_set.to_csv('../../submissions/model_infinity/teamCluster_submission_08.csv', index=True)
print("Completed!")

In [ ]:
submission_set['prediction'].idxmin()

In [ ]:
submission_set['prediction'].value_counts()